<a href="https://colab.research.google.com/github/CauchyLee1981/colab/blob/master/wordToText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


# 案１のデータGenerator

In [0]:
import os
import numpy as np
from keras.utils import np_utils


def my_generator(path="data", batch_size=50, n_classes=2, frame_num=32, shift=8):
    key_point=25
    channel=3
    
    if not os.path.exists(path):
        print(path, "is not exist\n")
        return

    train_data = []
    counter = 0
    for root, folders, _ in os.walk(path): break
    print(root,folders)
    while True:
      for folder in folders:
        for root2, _, files in os.walk(os.path.join(root, folder)): break
        print("\n",root2,files)
        for file in files:
          with open(os.path.join(root2, file)) as tmp_file:
            a = tmp_file.read()
            b = np.array(a.split()).reshape(-1, key_point, channel).astype(np.float32)
            for i in range(5):
              start = i * shift +frame_num
              end = start + frame_num
              X = b[start:end]
              #print(X.size)
              if not X.size == frame_num*key_point*channel:
                continue
              X1 = X.copy()
              X1[:-1, :, :] = X1[1:, :, :] - X1[:-1, :, :]
              X = X.reshape(1, frame_num, key_point, channel)
              X1 = X1.reshape(1, frame_num, key_point, channel)
              y = np_utils.to_categorical(int(folder), num_classes=n_classes)
              y = y.reshape(1, len(y))
              yield [X,X1], y



gen = my_generator(n_classes=100)
i = 0

for X,y in gen:
  #print(X)
  print(y)
  if i >= 50: 
    break
  else:
    i=i+1


# 案２のGenerator

## file list generator

In [0]:
import os
import numpy as np
from keras.utils import np_utils


def my_generator(path="data"):
    
    if not os.path.exists(path):
        print(path, "is not exist\n")
        return

    train_data = []
    counter = 0
    for root, folders, _ in os.walk(path): break
    print(root,folders)
    
    with open("listfile/SLR1_list_train.txt", mode='w') as train_f:
      with open("listfile/SLR1_list_test.txt", mode='w') as test_f:
        for folder in folders:
          for root2, _, files in os.walk(os.path.join(root, folder)): break
          print("\n",root2,files)
          for file in files:
            temp_str = folder + "," + os.path.join(root2, file) + "\n"
            # ファイル名に第三Underbarの後ろの数字を取得
            testflg=file.split('.',1)[0].split('_')[-1]
            print(testflg)
            if testflg is '4':
              test_f.write(temp_str)
            else:
              train_f.write(temp_str)


my_generator()


file list よりデータをgenerateする。

In [31]:
import os
import numpy as np
import random
from keras.utils import np_utils


def my_generator_1(list_path="listfile/SLR1_list_train.txt", batch_size=50, n_classes=2, frame_num=32, shift=8):
  key_point=25
  channel=3
  
  if not os.path.exists(list_path):
      print(path, "is not exist\n")
      return

  train_data = []
  counter = 0

  while True:
    with open(list_path) as list_f:
      files_list = list_f.readlines()
      random.shuffle(files_list)
      for line in files_list:
        #print(line)
        folder, file = line.split('\n')[0].split(',')
        #print(folder,file)
        with open(file) as tmp_file:
          a = tmp_file.read()
          b = np.array(a.split()).reshape(-1, key_point, channel).astype(np.float32)
          for i in range(5):
            start = i * shift +frame_num
            end = start + frame_num
            X = b[start:end]
            #print(X.size)
            if not X.size == frame_num*key_point*channel:
              continue
            X1 = X.copy()
            X1[:-1, :, :] = X1[1:, :, :] - X1[:-1, :, :]
            X = X.reshape(1, frame_num, key_point, channel)
            X1 = X1.reshape(1, frame_num, key_point, channel)
            y = np_utils.to_categorical(int(folder), num_classes=n_classes)
            y = y.reshape(1, len(y))
            yield [X,X1], y


gen = my_generator_1(n_classes=100)
i = 0

for X,y in gen:
  #print(X)
  print(y)
  if i >= 50: 
    break
  else:
    i=i+1

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

# ネットの定義

In [8]:
import time

import argparse
import json
import os

from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import *
from keras.layers.convolutional import *
from keras.layers.core import *
from keras.models import Model
from keras.optimizers import adam


def one_obj(frame_l=32, joint_n=25, joint_d=3, words_size=500):
    input_joints = Input(name='joints', shape=(frame_l, joint_n, joint_d))
    input_joints_diff = Input(name='joints_diff', shape=(frame_l, joint_n, joint_d))

    x = Conv2D(filters=64, kernel_size=(1, 1), padding='same')(input_joints)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # @LeakyReLU()(x)

    x = Conv2D(filters=32, kernel_size=(3, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)

    x = Permute((1, 3, 2))(x)

    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(x)
    x = LeakyReLU()(x)  # BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)

    x_d = Conv2D(filters=64, kernel_size=(1, 1), padding='same')(input_joints_diff)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)  # LeakyReLU()(x_d)

    x_d = Conv2D(filters=32, kernel_size=(3, 1), padding='same')(x_d)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)  # LeakyReLU()(x_d)

    x_d = Permute((1, 3, 2))(x_d)

    x_d = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x_d)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)  # LeakyReLU()(x_d)
    x_d = MaxPool2D(pool_size=(2, 2))(x_d)
    x_d = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(x_d)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)  # LeakyReLU()(x_d)
    x_d = MaxPool2D(pool_size=(2, 2))(x_d)

    x = concatenate([x, x_d], axis=-1)

    x = Conv2D(filters=128, kernel_size=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.1)(x)

    x = Conv2D(filters=256, kernel_size=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.1)(x)
    x = Flatten()(x)
    x = Dense(256)(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)
    x = Dense(words_size, activation='softmax')(x)
    return Model([input_joints, input_joints_diff], x)


model = one_obj(32, 25, 3, 100)
opt = adam(0.005, epsilon=1e-8)
model.compile(opt, 'categorical_crossentropy', metrics=['acc'])
model.summary()

early = EarlyStopping(monitor="loss", mode="min", patience=10)
lr_change = ReduceLROnPlateau(monitor="loss", verbose=1, factor=0.2, patience=100, min_lr=1e-8, cooldown=100)
lr_change.set_model(model)
lr_change.on_train_begin()
best_acc = 0.0



















Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
joints_diff (InputLayer)        (None, 32, 25, 3)    0                                            
__________________________________________________________________________________________________
joints (InputLayer)             (None, 32, 25, 3)    0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 32, 25, 64)   256         joints_diff[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 25, 64)   256      

# Train

In [32]:

train_generator = my_generator_1(n_classes=100)

# Train model on dataset
model.fit_generator(generator=train_generator,
          steps_per_epoch=256,
          epochs=1000)

Epoch 1/10
256/256 [==============================] - 24s 93ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 2/10
256/256 [==============================] - 22s 87ms/step - loss: 15.8033 - acc: 0.0195
Epoch 3/10
256/256 [==============================] - 22s 87ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 4/10
256/256 [==============================] - 24s 94ms/step - loss: 15.8033 - acc: 0.0195
Epoch 5/10
256/256 [==============================] - 22s 85ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 6/10
256/256 [==============================] - 23s 90ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 7/10
256/256 [==============================] - 32s 124ms/step - loss: 15.5514 - acc: 0.0352
Epoch 8/10
256/256 [==============================] - 24s 94ms/step - loss: 15.8663 - acc: 0.0156
Epoch 9/10
256/256 [==============================] - 21s 83ms/step - loss: 15.8033 - acc: 0.0195
Epoch 10/10
256/256 [==============================] - 24s 94ms/step - loss: 16.1181 - acc: 0.0000e+0